- optimizer /// 지각
- minmax 보다 z-score가 데이터의 왜곡이 적다

- CNN
    - Conv -  ReLu - Conv - ReLu - POOL x 3 + FullyConnected Layer
- Convolutional Layer
    - 특정 구간의 대표값을 뽑아내는 과정 (특징)
- Moving Filter
    - 전체 이미지에서 우리가 구간을 3x3이라고 정한다.
    - stride : 필터를 움직이는 범위
    - 7x7에서 진행할 경우 output = 5x5가 된다.
- Zero Padding:
    - 의미 없는 부분을 0으로 채워 넣는다.
    - 0은 검은색이지만 어떤 weight를 곱해도 0이므로 
    - Zero padding을 해도 stride =2 이면 이미지 사이즈가 줄어드는 것은 어쩔 수 없다.
    - stride =1 일 때 이미지 사이즈를 유지



- CNN 구현 예
    - 필터의 사이즈를 결정한다.
    - 필터의 stride를 결정한다.
    - zero-padding을 쓸지 말지를 결정한다.
    - 필터의 갯수를 결정한다. -> 필터의 갯수가 많아지면 이미지의 여러가지 특징을 도출할 가능성이 높아진다.
    - 필터의 각 값들은 랜덤하게 뽑아내므로 학습 결과가 같게 나오지 않는다. 
- POOLING Layer
    - 특징들을 추려내는 과정
    - 이미지를 줄인다.
    - Max Poolling : 
        - 각 필터의 결과로 나온 것에서 가장 큰 값을 도출한다.
        - stride, zero-padding 쓸지 말지 (추려내는 것이므로 일반적으로 zero-padding 사용 x)

In [ ]:
import tensorflow.compat.v1 as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib.pyplot as plt
tf.set_random_seed(777)

print(x_train.shape)
#x_train에는 이미지의 정보

In [ ]:
plt.imshow(x_train[0])
plt.show()
print(y_train.shape)
print(y_train[0])
#y_train에는 라벨이 들어가 있음을 확인할 수 있다.

In [ ]:
nb_class = 10 #0~9

X = tf.placeholder(tf.float32, [None,784]) #28x28 = 784
Y = tf.placeholder(tf.int32, [None, 1])

W = tf.Variable(tf.random_normal([784,nb_class]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_class]))

X_re = x_train.reshape(-1,784)
# X_r = tf.cast(X_re,tf.float32)
#print(X_r)
print(X_re.shape)
Y_one_hot = tf.one_hot(Y, nb_class)
# print(Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1,nb_class])
Y_t_re = y_train.reshape(-1,1)
print(Y_t_re.shape)
# print(Y_one_hot)
#print(y_train.shape)
# print(x_train.shape)


logits = tf.matmul(X,W) + b

hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost_i)

prediction = tf.argmax(hypothesis,1)

correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict = {X:X_re, Y:Y_t_re})
        if step % 400 ==0:
            loss, acc = sess.run([cost, accuracy],feed_dict = {X : X_re, Y:Y_t_re})
            print('step : {:5}\tLoss : {:.3f}\tAcc : {: .2%}'.format(step, loss, acc))
    pred = sess.run(prediction, feed_dict = {X : x_test.reshape(-1,784)})
    for p,y in zip(pred, y_test.flatten()):
#         print("[{}] Prediction : {} True Y : {}".format(p == int(y), p, int(y)))
        ind +=1
        if p != int(y):
            print("[{}] Prediction : {} True  Y :{}".format(p == int(y),p,int(y)))
            plt.imshow(x_test[ind])
            plt.show()

In [ ]:
#CNN
import tensorflow as tf
tf.__version__


In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)

mnist = input_data.read_data_sets('MNIST_data/',one_hot = True)

In [ ]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

X = tf.placeholder(tf.float32, [None,784])
X_img = tf.reshape(X, [-1,28,28,1]) #MNIST는 흑백이라서 1
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.palceholder
#stddev는 표준편차를 0.01로 줄인 것
#X_img와 맞춰주기 위해 3x3에 1 채널
#32는 필터의 갯수
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01)) 
#패딩사용 (SAME)
#더 고차원이 되면 필요한 strides의 4개의 파라미터
#현재는 strides에서 가운데 두개의 인자만 인식하면 됌 1,1만큼 움직이겠다
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob) #렐루에서 가져온 가중치를 70%만 활성화
#맥스풀에서 커널 사이즈는 2x2에 이동은 두칸씩한다는 의미이다.
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
#size = 14x14

#위에서 32개의 필터를 사용했으므로 L1(conv2d)의 결과 32겹의 필터가 생성되므로
#이것을 겹쳐서 depth = 32가 된다.
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01)) 
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding='SAME')
#size = 7x7

L2_flat = tf.reshape(L2, [-1, 7*7*64])
W3 = tf.Variable(tf.random_normal([7*7*64, 10]))
b = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(L2_flat , W3) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometimes.")
#epoch : 같은 데이터를 반복적으로 학습
#batch_size : 작으면 데이터 하나하나에 민감하기 때문에 Overfitting될 확률이 커진다.
# 야구 축구 농구 배구 같은 것을 분류하려면 세세한 데이터를 따져야 하므로 batch_size가 작은게 좋다
# 하지만 야구, 야구장, 야구공과 같은 것을 분류하는 것이라면 batch_size가 큰 것이 좋다.
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob : 0.7}
        c,_ = sess.run([cost, optimizer], feed_dict = feed_dict)
        avg_cost += c/total_batch
        
    print("Epoch :", '%04d' %(epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))
print("Learning Done")

correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy : ", sess.run(accuracy, feed_dict = \
                              {X : mnist.test.imgaes, Y: mnist.test.labels, keep_prob : 0.7}))
